# Attention
### seq2seq과의 차이
- seq2seq은 encoder에서 **context vector** 라고 부르는 **하나의 고정된 벡터 표현**으로 압축하고 decoder에서는 이 **context vector**를 이용하여 **output sequence**를 만들어냅니다.
  - 이러한 seq2seq은 2가지 문제를 갖습니다.
    - 고정된 하나의 context vector로 압축하는 과정에서 정보의 손실이 발생합니다.
    - RNN의 고질적 문제인 gradient vanish 문제가 발생합니다.
  - 이 2가지 문제는 input sentence가 길어질 때, quality of translation이 떨어지는 현상으로 나타납니다.
- Attention은 input sequence가 길어질 때에도 quality of output을 떨어뜨리지 않게 해주는 기법입니다.

### Attention IDEA
- Decoder에서 output을 예측하는 timestep마다 encoder에서의 전체 input sentence를 다시 참고
  - 이때 참고는 전체를 동일한 weight로 하는 것이 아닌 output과 연관이 있는 부분을 더 집중(Attention)하여 확인합니다.

### Attention Function
- Attention(Q,K,V) = Attention Value
  - Q : Query ( t 시점의 hidden state in decoder cell )
  - K : Keys ( 모든 시점의 hidden state in encoder cell )
  - V : Values ( 모든 시점의 hidden stae in encoder cell )
- Attention은 주어진 Q(Query)에 대해서 모든 K(Key)와의 유사도를 구합니다. 이 유사도를 K와 매핑된 V(Value)에 반영해줍니다. 그리고 유사도가 반영된 V를 모두 더하여 결과 Attention Value를 Return 합니다.


## Dot-Product Attention
- Attention에는 다양한 종류가 존재하는데 가장 수식적으로 쉬운 Attention 방식이 Dot-Product Attention입니다.
1. Attention Score 구하기
  - Decoder는 원래 t에서 t-1에서의 hidden state와 t-1에서의 output이 input으로 필요합니다.
  - Attention Mechanism에서는 추가적으로 output 예측을 위해 Attention value가 필요합니다.
    - 이 Attention value를 $a_t$ 라고 하겠습니다.
  - $a_t$를 구하기 위해서는 Attention Score를 구해야 합니다.
    - Attention Score는 encoder의 모든 hidden state 각각이 decoder 현 시점($t$)에서의 hidden state($s_t$)와 얼마나 유사한지를 판단하는 Score입니다.
    - 이것을 구하기 위해 Dot-Product Attention에서는 $s_t$를 transpose하고 각 encoder의 hidden state와 내적(Dot-Product)을 수행합니다. (내적이므로 결과는 스칼라입니다. )
    - 따라서 스코어 함수는 아래와 같습니다.
      - $score(s_t,h_i)={s^T_t}h_i$
    - 스코어 함수를 통해 구한 decoder의 $s_t$와 모든 encoder hidden state($h_i$)와의 스코어 모음 값들을 $e^t$라고 정의하겠습니다.
      - $e^t = [{s_^T_t}h_1, {s_^T_t}h_2,...,{s_^T_t}h_t] $
2. softmax를 이용하여 attention distribution을 구하기
    - $e^t$에 소프트 맥스 함수를 적용하면, 모든 값을 합하면 1이 되는 probability distribution을 얻을 수 있습니다.
      - 이 probability distribution을 attention distribution이라고 합니다.
        - 또한, 여기서 각각의 값을 Attention Weight라고 부릅니다.
      - Attention distribution을 $a^t$라고 부르고 식은 아래와 같습니다.
        - $α^t = softmax(e^t)$
3. Attention weight와 hidden state($h_i$)를 weighted sum 하여 Attention Value 구하기
  - Attention의 최종 결과를 얻기 위해 각 encoder의 hidden state($h_i$)와 Attention Weight들을 곱하고 더합니다. (Weighted Sum)
  - Attention의 최종 결과 식은 아래와 같습니다.
    - $a_t = {sum^N_{i=1}}{\alpha_i^t}h_i$
  - 이 Attention Value $a_t$는 Encoder의 Context를 포함하고 있다는 의미로 Context Vector라고 부릅니다.
    - seq2seq에서 encoder 마지막 hidden state를 context vector라고 부르는 것과 대조됩니다. ( Attention에서는 모든 Encoder의 hidden state를 weighted sum하기 때문입니다.)
4. Attention Value와 Decoder의 $t$시점의 hidden state($s_t$)를 concatenate
  - 3에서 계산한 $a_t$를 $s_t$와 concatenate하여 $v_t$를 구합니다.
    - 이 $v_t$는 $\hat{y}$ 예측 연산의 input으로 이용되어 encoder로부터 얻은 정보를 활용하여 $\hat{y}$ 예측을 잘 할 수 있게 합니다.
      - 이것이 Attention Mechanism 입니다.
        - 모든 Encoder의 정보를 weighted sum한 $a_t$를 이용하여 output 예측에 사용함으로서, output 예측 과정에서 모든 encoder의 hidden state를 참고하게 됩니다.
5. 출력층 연산의 입력이 되는 $\tilde{s_t}$를 계산합니다.
  - Attention Paper에서는 $v_t$를 바로 output layer로 보내지 않고 그 전에 신경망 연산을 하나 더 추가하였습니다.
    - 해당 연산은 아래와 같습니다.
      - $\tilde{s_t}= tanh(v_t*W_c + b_c)$
        - 내적이 아닌 곱연산입니다.
        - 여기서 $W_c$는 learnable weight matrix이며, $b_c$는 bias입니다.
6. 최종적으로 $\tilde{s_t}$를 output layer의 input으로 사용합니다.
  - 최종 결과 식은 아래와 같습니다.
    - $\hat{y_t} = Softmax(W_y{\tilde{s_t}}+ b_y)$


##다른 Attention
- Attention Score를 구하는 방식에서 Attention마다의 차이가 존재합니다.
1. dot-porduct (Luong Attention)
  - $score(s_t,h_i) = {s^T_t}{h_i}$
2. scaled-dot
  - $score(s_t,h_i) = \frac{{s^T_t}{h_i}}{\sqrt{n}}$
3. general
  - $score(s_t,h_i) = {s^T_t}{W_a}{h_i}$
4.  concat (Bahdanau Attemtion)
  - $score(s_t,h_i) = {W^T_a}tanh({W_b}[s_t;h_i])$
  - $score(s_t,h_i) = {W^T_a}tanh({W_b}{s_t}+{W_c}{h_i})$
5. location-base
  - $α_t = softmax({W_a}{s_t})$
    - $α_t$(attention value)산출 시에만 $s_t$를 사용하는 방식입니다.
- 여기서
  - $s_t$는 Query
  - $h_i$는 Key
  - $W_a$,$W_b$는 learnable weighted matrix


## Bahdanau Attention ( concat Attention )
### Bahdanau Attention 정의
- $Attention(Q,K,V) = Attention Value$
```
Q = Query : t-1 시점의 Decoder의 hidden state
K = Keys : 모든 시점의 Encoder의 hidden states
V = Values : 모든 시점의 Encoder의 hidden states
```

### Bahdanau Attention 연산
1. Attention Score 구하기
  - 우선, Bahdanau Attention에서는 encoder의 hidden states($h_1,h_2,...,h_t)$와 decoder의 hidden state($s_t$)가 같은 크기의 dimension을 가진다고 가정합니다.
  - 정의에 적혀있듯, Bahdanau Attention에서는 Dot-Product Attention과 다르게 Query로 t-1(한 단계 이전 시점)의 hidden state를 사용합니다.
    - 따라서 attention score 함수도 $s_{t-1}$을 사용하고 score함수는 아래와 같습니다.
      - $score(s_{t-1},h_i) ={W_a^T}tanh({W_b}{s_{t-1}}+W_c{h_i})$
        - 여기서 ${W_a},{W_b},{W_c}$는 모두 learnable weight matrix입니다.
        - 여기서 h_i는 여러 개이고 각각의 attention score를 구해야 하므로, 병렬 연산을 위해 하나의 행렬($H$)로 치환합니다.
        - $e^t = score(s_{t-1},H) ={W_a^T}tanh({W_b}{s_{t-1}}+W_c{H})$
2. Softmax를 이용하여 Attention distribution 구하기
  - Dot-Product Attention과 동일합니다.
3. Attention Weight와 hidden state ${s_{t-1}}$과 weighted sum을 통해 Attention Value 구하기
  - Dot-Product Attention과 동일합니다.
  - Attention Value = Context Vector
4. Context Vector로부터 ${s_t}$를 구합니다.
  - LSTM으로 돌아가 seq2seq에서 decoder로 사용한 LSTM 입력을 상기합니다.
    - LSTM은 입력으로 t-1 시점에서 전달받은 ${s_{t-1}}$과 t 시점에서의 입력 ${x_t}$를 이용하여 연산합니다.
  - Attention은 Context Vector와 ${x_t}$를 concatenate하여 새로운 t시점의 입력으로 사용합니다. 외에는 LSTM과 동일합니다.

# Attention 실습 ( Attention을 이용한 Translator )


### 1. Data Load and preprocessing
-  간단한 실습용 데이터를 이용합니다.
-  translator를 위해서는 parallel corpus 데이터가 필요합니다. ( 두 개 이상의 언어가 병렬적으로 구성된 corpus )
-  data를 다운 받은 후, 압축을 풀어 kor.txt 파일을 얻고 이것을 이용합니다.
  - [parallel corpus data link (kor-eng.zip)](http://www.manythings.org/anki)

In [40]:
# 1. Data Load and preprocessing
# 간단한 실습용 데이터를 이용합니다.
## translator를 위해서는 parallel corpus 데이터가 필요합니다. ( 두 개 이상의 언어가 병렬적으로 구성된 corpus )
## data를 다운 받은 후, 압축을 풀어 kor.txt 파일을 얻고 이것을 이용합니다.

## 1-1 Data Download
import re
import os
import unicodedata
import urllib3
import zipfile
import shutil
import numpy as np
import pandas as pd
import torch
from collections import Counter
from tqdm import tqdm
from torch.utils.data import DataLoader, TensorDataset

!wget -c https://www.manythings.org/anki/kor-eng.zip && unzip -o kor-eng.zip

--2025-02-13 16:12:09--  https://www.manythings.org/anki/kor-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

Archive:  kor-eng.zip
  inflating: _about.txt              
  inflating: kor.txt                 


In [41]:
## 1-2. preprocess function

#사용할 샘플 수를 정합니다.
num_samples = 4444

def en_preprocess_sentence(sentence):
  sent = re.sub(r"([?.!,?`])"," \1",sentence) #단어와 구두점 사이에 공백을 만듭니다.
  sent = re.sub(r"[^a-zA-Z!.?]+",r" ",sent) # 결과에서 a-z,A-Z,!,.,?를 제외하고 모두 공백으로 변환합니다.
  sent = re.sub(r"\s+"," ",sent) #공백이 여러 개인 경우 하나로 변환합니다.
  return sent

def ko_preprocess_sentence(sentence):
  sent = re.sub(r"([?.!,?`])"," \1",sentence) #단어와 구두점 사이에 공백을 만듭니다.
  sent = re.sub(r"[^ㄱ-힣!.?]+",r" ",sent) # 결과에서 ㄱ-ㅎ,!,.,?를 제외하고 모두 공백으로 변환합니다.
  sent = re.sub(r"\s+"," ",sent) #공백이 여러 개인 경우 하나로 변환합니다.
  return sent

def load_preprocessed_data():
  encoder_input,decoder_input,decoder_target = [],[],[]
  with open("kor.txt","r") as lines :
    for i,line in enumerate(lines):
      src_line, tar_line, _ = line.strip().split("\t") # 데이터는 소스와 타겟이 탭으로 구분되어 있습니다. # _는 split을 하는 경우 뒤에 주석까지 따라오기에 처리하기 위한 부분입니다.
      src_line = [ w for w in en_preprocess_sentence(src_line).split()]

      tar_line = ko_preprocess_sentence(tar_line)
      tar_line_in = [w for w in ("<sos> "+ tar_line).split()]
      tar_line_out = [w for w in (tar_line + " <eos>").split()]

      encoder_input.append(src_line)
      decoder_input.append(tar_line_in)
      decoder_target.append(tar_line_out)
      if i== num_samples - 1 :
        break
  return encoder_input,decoder_input,decoder_target

In [42]:
# 전처리 함수를 테스트합니다.
en = "Women like men with mustaches."
ko = "여성은 수염이 있는 남성을 좋아해."
print("전처리 후 en :",en_preprocess_sentence(en))
print("전처리 후 ko :",ko_preprocess_sentence(ko))

전처리 후 en : Women like men with mustaches 
전처리 후 ko : 여성은 수염이 있는 남성을 좋아해 


In [43]:
# 데이터셋을 불러옵니다.
sent_en_in,sent_ko_in,sent_ko_out = load_preprocessed_data()
print('encoder input :',sent_en_in[:10])
print('decoder input :',sent_ko_in[:10])
print('decoder label :',sent_ko_out[:10])
"""
여기에서 앞에서 공부한 것과 다르게 이전 시점의 decoder 셀의 출력을 넣어주는 것이 아닌 이전 시점의 실제 출력 값을 넣어주는 방법을 사용합니다.
이 이유는 만약 훈련 과정에서 이전 decoder cell의 출력 결과가 잘못되었는데 그것을 이용하여 출력한다면 연쇄적으로 잘못된 값을 예측하며 전체적으로 훈련 시간이 길어집니다.
이 상황을 해결하기 위해 실제 값을 넣어주는 이러한 방식을 "교사 강요" 라고 합니다.
"""

encoder input : [['Go'], ['Hi'], ['Run'], ['Run'], ['Who'], ['Wow'], ['Duck'], ['Fire'], ['Help'], ['Hide']]
decoder input : [['<sos>', '가'], ['<sos>', '안녕'], ['<sos>', '뛰어'], ['<sos>', '뛰어'], ['<sos>', '누구'], ['<sos>', '우와'], ['<sos>', '숙여'], ['<sos>', '쏴'], ['<sos>', '도와줘'], ['<sos>', '숨어']]
decoder label : [['가', '<eos>'], ['안녕', '<eos>'], ['뛰어', '<eos>'], ['뛰어', '<eos>'], ['누구', '<eos>'], ['우와', '<eos>'], ['숙여', '<eos>'], ['쏴', '<eos>'], ['도와줘', '<eos>'], ['숨어', '<eos>']]


'\n여기에서 앞에서 공부한 것과 다르게 이전 시점의 decoder 셀의 출력을 넣어주는 것이 아닌 이전 시점의 실제 출력 값을 넣어주는 방법을 사용합니다.\n이 이유는 만약 훈련 과정에서 이전 decoder cell의 출력 결과가 잘못되었는데 그것을 이용하여 출력한다면 연쇄적으로 잘못된 값을 예측하며 전체적으로 훈련 시간이 길어집니다.\n이 상황을 해결하기 위해 실제 값을 넣어주는 이러한 방식을 "교사 강요" 라고 합니다.\n'

In [44]:
len(sent_en_in)

4444

In [45]:
# 단어 집합을 만들어 단어로부터 정수를 얻도록 합니다.
def build_vocab(sents):
  word_list = []
  for sent in sents : #전달받은 문장들 중 하나씩 가져옵니다.
    for word in sent : # 문장들의 단어를 하나씩 가져옵니다.
      word_list.append(word)
  # 각 단어 별 등장 빈도에 따라 정렬합니다.
  word_counts = Counter(word_list)
  vocab = sorted(word_counts,key = word_counts.get, reverse = True) #word_counts를 역순으로 정렬합니다.
  word_to_index = {}
  word_to_index['<PAD>'] =0
  word_to_index['<UNK>'] =1

  for index, word in enumerate(vocab):
    word_to_index[word] = index + 2 # PAD, UNK가 0,1 인덱스를 가지기 때문
  return word_to_index

In [46]:
# en, ko 각각에 대해 vocab을 만듭니다.
en_vocab = build_vocab(sent_en_in)
ko_vocab = build_vocab(sent_ko_in+sent_ko_out)

# 각 size(단어 개수)를 확인합니다.
print("en size :",len(en_vocab))
print("ko size :",len(ko_vocab))

# 내용을 확인합니다.
print("en keys (3~5) :" ,list(en_vocab.keys())[3:6])
print("en values (3~5) :" ,list(en_vocab.values())[3:6])
print("ko keys (3~5) :" ,list(ko_vocab.keys())[3:6])
print("ko values (3~5) :" ,list(ko_vocab.values())[3:6])

en size : 2602
ko size : 5476
en keys (3~5) : ['Tom', 'is', 'you']
en values (3~5) : [3, 4, 5]
ko keys (3~5) : ['<eos>', '톰은', '나는']
ko values (3~5) : [3, 4, 5]


In [47]:
# {word : index} 형태로 변환합니다.
index_to_src = {v: k for k,v in en_vocab.items()}
index_to_tar = {v: k for k,v in ko_vocab.items()}

def texts_to_sequences(sents, word_to_index):
  encoded_X_data = []
  for sent in tqdm(sents):
    index_sequences = []
    for word in sent:
      try :
        index_sequences.append(word_to_index[word])
      except : # 없는 단어라면 UNK 추가
        index_sequences.append(word_to_index['<UNK>'])
    encoded_X_data.append(index_sequences)
  return encoded_X_data

In [48]:
encoder_input = texts_to_sequences(sent_en_in,en_vocab)
decoder_input = texts_to_sequences(sent_ko_in,ko_vocab)
decoder_target = texts_to_sequences(sent_ko_out,ko_vocab)

for i,(item1,item2) in zip(range(5),zip(sent_en_in,encoder_input)):
  # encoder input과 english sentence 앞의 5개를 통해 정수화가 잘 되는지 확인합니다.
  print(f"Index: {i}, 정 수 인 코 딩 전 : {item1}, 정 수 인 코 딩 후 : {item2}")

100%|██████████| 4444/4444 [00:00<00:00, 560434.38it/s]

Index: 0, 정 수 인 코 딩 전 : ['Go'], 정 수 인 코 딩 후 : [281]
Index: 1, 정 수 인 코 딩 전 : ['Hi'], 정 수 인 코 딩 후 : [1470]
Index: 2, 정 수 인 코 딩 전 : ['Run'], 정 수 인 코 딩 후 : [1004]
Index: 3, 정 수 인 코 딩 전 : ['Run'], 정 수 인 코 딩 후 : [1004]
Index: 4, 정 수 인 코 딩 전 : ['Who'], 정 수 인 코 딩 후 : [89]


In [49]:
# 길이 일치를 위한 패딩 함수를 구현합니다.
def pad_sequences(sentences, max_len=None):
  if max_len is None :
    max_len = max([len(sentence) for sentence in sentences])
  features = np.zeros((len(sentences),max_len),dtype=int)
  for index,sentence in enumerate(sentences):
    if len(sentence) != 0:
      features[index,:len(sentence)] = np.array(sentence)[:max_len]
  return features

In [50]:
# 패딩으로 전체 길이를 일치시킵니다.
encoder_input = pad_sequences(encoder_input)
decoder_input = pad_sequences(decoder_input)
decoder_target = pad_sequences(decoder_target)

In [51]:
# 데이터 shape을 확인합니다. (1 차이는 <sos> <eos> 입니다.)
print(' 인 코 더 의 입 력 의 크 기 (shape) :',encoder_input.shape)
print(' 디 코 더 의 입 력 의 크 기 (shape) :',decoder_input.shape)
print(' 디 코 더 의 레 이 블 의 크 기 (shape) :',decoder_target.shape)

 인 코 더 의 입 력 의 크 기 (shape) : (4444, 9)
 디 코 더 의 입 력 의 크 기 (shape) : (4444, 10)
 디 코 더 의 레 이 블 의 크 기 (shape) : (4444, 10)


In [52]:
# 테스트 데이터 분리 전, 데이터를 섞습니다.
indices = np.arange(encoder_input.shape[0]) # 전체 개수만큼 리스트를 생성합니다.
np.random.shuffle(indices) # 섞습니다.
encoder_input = encoder_input[indices] # idx를 이용하여 순서를 섞습니다.
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

In [53]:
# 데이터를 분할합니다. 6:2:2
test_size = int(num_samples*0.1)
train_size = num_samples-test_size*2 # val, test 각각 빼줍니다.

encoder_input_train = encoder_input[:train_size]
encoder_input_test = encoder_input[train_size:train_size+test_size]
encoder_input_val = encoder_input[train_size+test_size:]

decoder_input_train = decoder_input[:train_size]
decoder_input_test = decoder_input[train_size:train_size+test_size]
decoder_input_val = decoder_input[train_size+test_size:]

decoder_target_train = decoder_target[:train_size]
decoder_target_test = decoder_target[train_size:train_size+test_size]
decoder_target_val = decoder_target[train_size+test_size:]

In [54]:
# 분할 후 size를 확인합니다.
print('train source data :',encoder_input_train.shape)
print('train target data :',decoder_input_train.shape)
print('train target label :',decoder_target_train.shape)
print('test source data :',encoder_input_test.shape)
print('test target data :',decoder_input_test.shape)
print('test target label :',decoder_target_test.shape)

train source data : (3556, 9)
train target data : (3556, 10)
train target label : (3556, 10)
test source data : (444, 9)
test target data : (444, 10)
test target label : (444, 10)


### 2. machine translator 제작

In [55]:
import torch
import torch.nn as nn
import torch.optim as optim

embedding_dim =256
hidden_units = 256

In [56]:
# Encoder, seq2seq과 구조는 동일합니다.
class Encoder(nn.Module):
  def __init__(self,src_vocab_size,embedding_dim,hidden_units):
    super(Encoder,self).__init__()
    self.embedding = nn.Embedding(src_vocab_size,embedding_dim,padding_idx=0)
    self.lstm = nn.LSTM(embedding_dim,hidden_units,batch_first=True)
  def forward(self,x):
    # x.shape == (batchsize,seq_len,embedding_dim)
    x = self.embedding(x)
    # hidden.shape == (1,batch_size,hidden_units), cell.shape == (1,batch_size,hidden_units)
    outputs, (hidden,cell) = self.lstm(x)
    return outputs,hidden,cell

In [57]:
# Decoder (Luong Attention)
# s_t와 모든 h_i와의 dot-product를 이용하여 attention score를 구합니다.
# 이후, attention score를 softmax 함수를 통과시켜, attention_weights를 얻습니다.
# attention weights는 h_i들과 곱한 후, 더해 context vector를 얻습니다.
# context vector를 활용하는 방식은 다양한데 이 코드에선, embedding vector와 연결되어 input으로 활용합니다.

class Decoder(nn.Module):
  def __init__(self, tar_vocab_size,embedding_dim,hidden_units):
    super(Decoder,self).__init__()
    self.embedding = nn.Embedding(tar_vocab_size,embedding_dim,padding_idx=0)
    self.lstm = nn.LSTM(embedding_dim+hidden_units,hidden_units,batch_first=True)
    self.fc = nn.Linear(hidden_units,tar_vocab_size)
    self.softmax = nn.Softmax(dim = 1)
  def forward(self, x, encoder_outputs, hidden, cell):
    x = self.embedding(x)
    # bmm = batch matix multiplication
    # attention scores = encoder의 모든 hidden state들과 decoder의 현재 시점 hidden state의 dot-product
    # scores.shape == (batch_size,source_seq_len,1)
    attention_scores = torch.bmm(encoder_outputs,hidden.transpose(0,1).transpose(1,2))
    # attention weighs = attention score의 softmax
    attention_weights = self.softmax(attention_scores)
    # context vector = encoder의 모든 hidden states와의 weighted sum
    context_vector = torch.bmm(attention_weights.transpose(1,2),encoder_outputs)
    # context vector.shape == (batch_size,1,hidden_units)
    # context vector isn't match second dimension of x. so, we need to extend length of context vector using repeat method for concatenating
    seq_len = x.shape[1]
    context_vector_repeated = context_vector.repeat(1,seq_len,1)
    # x.shape : (batch_size, target_seq_len, embedding_dim+hidden_unis)
    x = torch.cat((x,context_vector_repeated),dim=2)

    # output.shape : (batch_size, target_seq_len, hidden_unis)
    # hidden.shape : (1, batch_size, hidden_units)
    # cell.shape : (1, batch_size, hidden_units)
    output, (hidden,cell) = self.lstm(x,(hidden,cell))
    # output.shape : (batch_size, target_seq_len, tar_vocab_size)
    output = self.fc(output)

    return output,hidden,cell

In [58]:
# seq2seq model을 제작합니다. encoder와 decoder를 연결하는 형태입니다.
# attention 자체는 seq2seq의 long sequence에서 발생하는 문제를 보정하기 위한 방법론으로 제작되었기에
# decoder class 내부의 코드가 attention mechanism을 정의하는 것입니다.
class Seq2Seq(nn.Module):
  def __init__(self,encoder,decoder):
    super(Seq2Seq,self).__init__()
    self.encoder = encoder
    self.decoder = decoder
  def forward(self,src,trg):
    encoder_outputs, hidden, cell = self.encoder(src)
    output,_,_ = self.decoder(trg,encoder_outputs, hidden, cell)
    return output

encoder = Encoder(len(en_vocab),embedding_dim,hidden_units)
decoder = Decoder(len(ko_vocab),embedding_dim,hidden_units)
model = Seq2Seq(encoder,decoder)

loss = nn.CrossEntropyLoss(ignore_index = 0)
optimizer = optim.Adam(model.parameters())

In [59]:
# evaluation function
def evaluation(model,dataloader,loss_function, device):
  model.eval()
  total_loss = 0.0
  total_correct = 0
  total_count = 0
  with torch.no_grad():
    for encoder_inputs, decoder_inputs, decoder_targets in dataloader:
      encoder_inputs = encoder_inputs.to(device)
      decoder_inputs = decoder_inputs.to(device)
      decoder_targets = decoder_targets.to(device)

      #forward
      # output.shape == (batch_size, seq_len, tar_vocab_size)
      outputs = model(encoder_inputs, decoder_inputs)

      # loss 계산
      # outputs.view(-1,outputs.size(-1)).shape == (batch_size * seq_len, tar_vocab_size)
      # decoder_targets.view(-1).shape == (batch_size * seq_len)
      total_loss += loss(outputs.view(-1,outputs.size(-1)),decoder_targets.view(-1)).item()

      # acc 계산
      mask = decoder_targets != 0
      total_correct += ((outputs.argmax(dim=-1)==decoder_targets)*mask).sum().item()
      total_count += mask.sum().item()

  return total_loss / len(dataloader), total_correct / total_count

In [60]:
# 각 데이터를 tensor로 변환하고 batch_size 128로 설정합니다.

encoder_input_train_tensor = torch.tensor(encoder_input_train, dtype = torch.long)
decoder_input_train_tensor = torch.tensor(decoder_input_train, dtype = torch.long)
decoder_target_train_tensor = torch.tensor(decoder_target_train, dtype = torch.long)

encoder_input_test_tensor = torch.tensor(encoder_input_test, dtype=torch.long)
decoder_input_test_tensor = torch.tensor(decoder_input_test, dtype=torch.long)
decoder_target_test_tensor = torch.tensor(decoder_target_test, dtype=torch.long)
# 데이터셋 및 데이터로더 생성
batch_size = 128
train_dataset = TensorDataset(encoder_input_train_tensor, decoder_input_train_tensor, decoder_target_train_tensor)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_dataset = TensorDataset(encoder_input_test_tensor, decoder_input_test_tensor , decoder_target_test_tensor)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)  # 학습 설정

# epoch
epochs = 70
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(2602, 256, padding_idx=0)
    (lstm): LSTM(256, 256, batch_first=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(5476, 256, padding_idx=0)
    (lstm): LSTM(512, 256, batch_first=True)
    (fc): Linear(in_features=256, out_features=5476, bias=True)
    (softmax): Softmax(dim=1)
  )
)

In [62]:
best_val_acc = float(0)

for epoch in range(epochs):
  model.train()
  for encoder_inputs,decoder_inputs,decoder_targets in train_dataloader:
    encoder_inputs = encoder_inputs.to(device)
    decoder_inputs = decoder_inputs.to(device)
    decoder_targets = decoder_targets.to(device)

    optimizer.zero_grad()

    # forward
    outputs = model(encoder_inputs,decoder_inputs)

    # loss & backward
    loss_value = loss(outputs.view(-1,outputs.size(-1)),decoder_targets.view(-1))
    loss_value.backward()

    # propagation
    optimizer.step()
  train_loss,train_acc = evaluation(model,train_dataloader,loss,device)
  val_loss,val_acc = evaluation(model,valid_dataloader,loss,device)
  print(f'Epoch: {epoch+1}/{epochs} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Valid Loss: {val_loss:.4f} | Valid Acc: { val_acc:.4f}')

  if val_acc > best_val_acc :
    best_val_acc = val_acc
    print("best checkpoint is saved")
    torch.save(model.state_dict(),'best_model.pth')

KeyboardInterrupt: 

In [63]:
# best model load loss,acc 확인
model.load_state_dict(torch.load('best_model.pth'))
model.to(device)

val_loss,val_cc = evaluation(model,valid_dataloader,loss,device)

print("Best loss :",val_loss)
print("Best acc :",val_acc)

<ipython-input-63-824cfedb7421>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))


Best loss : 5.9084848165512085
Best acc : 0.3721881390593047


### seq2seq 기계 번역기 동작
- 훈련 과정에서 **교사 강요** 방식을 사용했기에 테스트 과정에서는 동작 방식이 다릅니다.
  - 따라서 새로 설계해줍니다.
- 번역 단계
  1. input sentence가 encoder에 입력, encoder의 마지막 시점의 hidden_state($h_t$)와 cell state를 얻습니다.
  2. $h_t$, cell state, \<sos\>를 decoder로 보냅니다.
  3. decoder가 \<eos\>를 출력할 때까지 다음 예측을 반복합니다.
  

In [64]:
# 정수를 단어로 변환하는 함수를 작성합니다.
index_to_src = {v: k for k,v in en_vocab.items()}
index_to_tar = {v: k for k,v in ko_vocab.items()}

def seq_to_src(input_seq):
  sentence = ''
  for encoded_word in input_seq:
    if (encoded_word != 0):
      sentence = sentence + index_to_src[encoded_word] + ' '
  return sentence

def seq_to_tar(input_seq):
  sentence = ''
  for encoded_word in input_seq:
    if (encoded_word != 0) and (encoded_word != ko_vocab['<sos>']) and(encoded_word != ko_vocab['<eos>']):
      sentence = sentence + index_to_tar[encoded_word] + ' '
  return sentence


In [65]:
def decode_sequence(input_seq,model,src_vocab_size,tar_vocab_size,max_output_len, int_to_src_token, int_to_tar_token):
  encoder_inputs = torch.tensor(input_seq,dtype=torch.long).unsqueeze(0).to(device)

  # set encoder initial state
  encoder_outputs, hidden, cell = model.encoder(encoder_inputs)

  # <sos>를 decoder의 first input으로 설정
  # unsqueeze는 batch 차원을 위해 추가합니다.
  decoder_input = torch.tensor([3],dtype=torch.long).unsqueeze(0).to(device)

  decoded_tokens = []

  # for 문을 반복하며 decoder가 반복 예측하도록 합니다.
  for _ in range(max_output_len):
    output,hidden,cell = model.decoder(decoder_input,encoder_outputs,hidden,cell)

    # 소프트맥스 회귀를 수행, 예측 단어의 인덱스
    output_token = output.argmax(dim=-1).item()

    # 종료 토큰 <eos>
    if output_token == 4:
      break

    # 각 시점의 단어(정수)는 decoded_tokens에 누적 후, 최종 번역을 진행합니다.
    decoded_tokens.append(output_token)

    # 현재 시점의 예측. 다음 시점의 입력으로 사용된다.
    decoder_input = torch.tensor([output_token], dtype=torch.long).unsqueeze(0).to(device)
  return ' '.join(int_to_tar_token[token] for token in decoded_tokens)

In [66]:
# 테스트 결과를 확인합니다.

for seq_index in [1123,2234,2567,1890,2353]:
  input_seq = encoder_input_train[seq_index]
  translated_text = decode_sequence(input_seq,model,len(en_vocab),len(ko_vocab),30,index_to_src,index_to_tar)

  print("input sentence :",seq_to_src(encoder_input_train[seq_index]))
  print("correct sentence :",seq_to_tar(decoder_input_train[seq_index]))
  print("output sentence :",translated_text)
  print("--"*50)

input sentence : Let the bird fly away 
correct sentence : 새가 날아가도록 풀어주자 
output sentence : 그 남자 바빠 <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos>
----------------------------------------------------------------------------------------------------
input sentence : Everybody left 
correct sentence : 모두 떠났어 
output sentence : 모두 떠났어 <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos>
----------------------------------------------------------------------------------------------------
input sentence : She overslept 
correct sentence : 그 사람은 늦잠잤어 
output sentence : 그 사람은 늦잠잤어 <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos>
-----------------------------